In [1]:
import kfp
import kfp.dsl as dsl

BASE_IMAGE = 'tensorflow/tensorflow:latest-gpu'

In [3]:
@dsl.component(
    base_image=BASE_IMAGE,
)
def check_gpu_access():
    """
    A simple component that checks and prints GPU availability using TensorFlow.
    """
    import tensorflow as tf
    import logging # Use logging for better output in KFP

    logging.basicConfig(level=logging.INFO)

    try:
        gpus = tf.config.list_physical_devices('GPU')
        if not gpus:
            print("TensorFlow CANNOT see any GPUs. GPU acceleration is NOT possible.")
        else:
            print(f"TensorFlow found {len(gpus)} GPU(s): {gpus}")

    except Exception as e:
        logging.error(f"An error occurred while checking for GPUs: {e}")
        raise e

In [4]:
@dsl.pipeline(
    name='gpu-test-pipeline',
    description='A minimal pipeline to test GPU access in a component.'
)
def gpu_test_pipeline():
    """Defines the pipeline structure."""
    check_gpu_task = (
        check_gpu_access()
        .set_display_name("Check GPU Availability")
        .set_gpu_limit(1)
        .add_node_selector_constraint('nvidia.com/gpu')
        .set_memory_limit("2G")
        .set_memory_request("1G")
        .set_cpu_limit("1")
        .set_cpu_request("0.5")
    )

In [5]:
client = kfp.Client()
client.create_run_from_pipeline_func(
    gpu_test_pipeline,
    experiment_name="mnist_pipeline",
)

/opt/conda/lib/python3.11/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


RunPipelineResult(run_id=92c6d5bf-bb86-406b-807c-1db9b513f8d0)